In [2]:
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import pauli_str_to_matrix

# 首先生成泡利字符串表示下的哈密顿量
# 相当于0.4*kron(I, Z) + 0.4*kron(Z, I) + 0.2*kron(X, X)
# 其中， X，Y, Z是泡利矩阵， I是单位矩阵
H_info = [[0.4, 'z0'], [0.4, 'z1'], [0.2, 'x0,x1']]

# 超参数设置
num_qubits = 2
theta_size = 4
ITR = 60
LR = 0.4
SEED = 999       

# 把记录的关于哈密顿量的信息转化为矩阵表示
H_matrix = pauli_str_to_matrix(H_info, num_qubits)
class vqe_demo(paddle.nn.Layer):
    
    def __init__(self, shape, dtype='float64'):
        super(vqe_demo, self).__init__()
        
        # 初始化一个长度为theta_size的可学习参数列表，并用[0, 2*pi]的均匀分布来填充初始值
        self.theta = self.create_parameter(shape=shape, 
                                           default_initializer=paddle.nn.initializer.Uniform(low=0., high=2*np.pi), 
                                           dtype=dtype, is_bias=False)
        self.H = paddle.to_tensor(H)
        
    # 定义损失函数和前向传播机制
    def forward(self):
        
        # 初始量子电路
        cir = UAnsatz(num_qubits)
        
        # 添加量子门
        cir.ry(self.theta[0], 0)
        cir.ry(self.theta[1], 1)
        cir.cnot([0, 1])
        cir.ry(self.theta[2], 0)
        cir.ry(self.theta[3], 1)
        
        # 选择用量子态的向量表示
        cir.run_state_vector()
        
        # 计算当前量子态下关于观测量H_info的期望值
        # 也就是 <psi|H|psi>
        loss = cir.expecval(H_info)
        
        return loss
loss_list = []
parameter_list = []

# 定义网络维度
vqe = vqe_demo([theta_size])

# 一般来说，我们利用Adam优化器来获得相对好的收敛，当然你可以改成SGD或者是RMS prop.
opt = paddle.optimizer.Adam(learning_rate = LR, parameters = vqe.parameters())    

# 优化循环
for itr in range(ITR):

    # 前向传播计算损失函数
    loss = vqe()

    # 反向传播极小化损失函数
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    # 记录学习曲线
    loss_list.append(loss.numpy()[0])
    parameter_list.append(vqe.parameters()[0].numpy())
    if itr % 10 == 0:
        print('iter:', itr, '  loss: %.4f' % loss.numpy())


print('计算得到的基态能量是: ', loss_list[-1])
print('真实的基态能量为: ', np.linalg.eigh(H_matrix)[0][0])

NameError: name 'H' is not defined

In [ ]:
import numpy as np
A = np.array([[1,2,3],[0,1,4],[0,0,3]])
x = np.array([1,1,1])
c = A.dot(x)
print(c)
d = x.transpose().dot(A.transpose())
print(d)
e = np.array([[1],[2]])
print(e)

In [ ]:
Id = np.identity(2)
Z = np.array([[1, 0], [0, -1]])
X = np.array([[0, 1], [1, 0]])

A_0 = np.identity(8)
A_1 = np.kron(np.kron(X, Z), Id)
A_2 = np.kron(np.kron(X, Id), Id)

A_num = c[0] * A_0 + c[1] * A_1 + c[2] * A_2
b = np.ones(8) / np.sqrt(8)
print("A = \n", A_num)
print("b = \n", b)
A_inv = np.linalg.inv(A_num)
print('Ainv = \n',A_inv)
# c = np.dot(A_num,A_inv)
# print('c = \n ',c)
# x = np.dot(A_inv, b)
# c_probs = (x / np.linalg.norm(x)) ** 2
# d = np.dot(A_num,x)